## Data Moudle 만들기

7에서 만든 모델 가져오기

In [4]:

import pytorch_lightning as pl
import torch
from torch import nn
from torchinfo import summary

from torchmetrics import functional as FM


class Model(pl.LightningModule):
    
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 32)
        self.linear2 = nn.Linear(28*28, 32)
        self.linear3 = nn.Linear(32+32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.flatten(x)

        x1 = self.linear1(x)
        x1 = self.relu(x1) 
        
        x2 = self.linear2(x)
        x2 = self.relu(x2)

        x3 = torch.cat([x1, x2], dim=1)
        y = self.linear3(x3) 
        
        return(y)



loss_ftn = nn.CrossEntropyLoss()
class MyModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.layers = Model()  ## 1개의 모델을 통으로 가지고 있는 형태 (상속이랑은 미묘하게 다름 -> 여러개 모델을 동시에 거느리는걸 생각해보자)

    
    def forward(self, x):
        out = self.layers(x)
        return(out)


    def predict_step(self, x, batch_idx):  ## pred 에서는 x 만 들어오기 때문에 batch 대신 x 라고 표시
        y_pred = self(x) # 여기까진  logit 
        y_prob = nn.Softmax(y_pred)  # 확률로 변환 
        return(y_prob)

     
    def training_step(self, batch, batch_idx): ## 학습시에는 (x, y) 쌍이 들어오므로 batch 라고 표현
        x, y = batch
        y_pred = self(x)
        loss = loss_ftn(y_pred, y)

        acc = FM.accuracy(y_pred, y, task = 'multiclass', num_classes = 10) 
        mse = FM.mean_squared_error( torch.argmax( y_pred, dim=1 ), y)

        metrics = {'loss' : loss, 'acc' : acc, 'mse' : mse }
        self.log_dict( metrics, prog_bar = True) 

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_ftn(y_hat, y)    
        acc = FM.accuracy(y_hat, y, task = 'multiclass', num_classes = 10) 
        mse = FM.mean_squared_error( torch.argmax( y_hat, dim=1 ), y)
        metrics = {'val_loss' : loss, 'val_acc' : acc, 'val_mse' : mse }
        self.log_dict( metrics, prog_bar = True) 
        return 
        

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_ftn(y_hat, y)    
        acc = FM.accuracy(y_hat, y, task = 'multiclass', num_classes = 10) 
        mse = FM.mean_squared_error( torch.argmax( y_hat, dim=1 ), y)
        metrics = {'test_loss' : loss, 'test_acc' : acc, 'test_mse' : mse }
        self.log_dict( metrics, prog_bar = True) 
        return 


    def configure_optimizers(self):
        return torch.optim.Adam( self.parameters(), lr=0.001 )






In [5]:
from torchvision.datasets import MNIST
import pytorch_lightning as pl
import torchvision.transforms as transforms
from torch.utils import data
from torch.utils.data import DataLoader

토치 라이트닝에서 제공하는 데이터 모듈은 크게 다음 매쏘드들만 정의하면 사용할 수 있다.
https://lightning.ai/docs/pytorch/stable/data/datamodule.html

- prepare
- setup
- train_dataloder
- val_dataloader
- test_dataloader


prepare 와 setup 은 데이터를 준비하는 과정이지만 prepare 는 1개의 single thread로 작동하기 때문에, 다운로드 같이 완결성이 필요한 데서 사용하고 setup 의 경우에는 prepare 이후에 로드가 되며, GPU 별로 실행되는 부분이 들어가면 된다고 한다. (데이터의 변환 등)

In [6]:
class MNIST_DataModule( pl.LightningDataModule ):

    def __init__(self, data_dir : str = '', batch_size :int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size


## 이미 다운로드가 받아져있는 데이터를 사용한다고 가정했으므로 prepare 생략 가능
    
    
    def setup(self, stage):

        #transforms for images
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))  ## 0~255 를  평균 / 산포 값으로 normalze
        ])

        train_data = MNIST(self.data_dir, train=True, transform=transform, download=True)   ## train과 test를 각각 변환
        test_data = MNIST(self.data_dir, train=False, transform=transform, download=True)
        
        self.mnist_test = test_data
        self.mnist_train, self.mnist_valid = data.random_split( train_data, [50000, 10000] ) 


   ## 데이터를 내보낼 때에는 torch.util 의 dataloader 로 내보낸다. 
    
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size = self.batch_size)  ## train 에 사용될 데이터 보내기

    def val_dataloader(self):
        return DataLoader(self.mnist_valid, batch_size = self.batch_size)  ## valid 에 사용될 데이터 보내기

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size = self.batch_size)  ## test 에 사용될 데이터 보내기 
        

In [7]:
data_module = MNIST_DataModule(batch_size= 128)

In [8]:
model = MyModel()

In [9]:
trainer = pl.Trainer(max_epochs=5, accelerator='auto')
trainer.fit(model, data_module)  ## 라이트닝에서 제일 일반적으로 학습을 시키는 형태 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\msong\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type  | Params
---------------------------------
0 | layers | Model | 50.9 K
---------------------------------
50.9 K    Trainable params
0         Non-trainable params
50.9 K    Total params
0.204     Total estimated model param

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\msong\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\msong\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [10]:
trainer.validate(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.validating metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.9758999943733215
        val_loss            0.0886077806353569
         val_mse            0.4876999855041504
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.0886077806353569,
  'val_acc': 0.9758999943733215,
  'val_mse': 0.4876999855041504}]

In [11]:
trainer.test(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\msong\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9682000279426575
        test_loss           0.10530847311019897
        test_mse            0.6162999868392944
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.10530847311019897,
  'test_acc': 0.9682000279426575,
  'test_mse': 0.6162999868392944}]